In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv(r"C:\Users\gagan soni\Downloads\penguins_size_decition_tree_classifier.csv")
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


# Preprocessing

In [3]:
df.sex.replace('.',df.sex.mode()[0],inplace = True)

In [4]:
df.isnull().sum()

species               0
island                0
culmen_length_mm      2
culmen_depth_mm       2
flipper_length_mm     2
body_mass_g           2
sex                  10
dtype: int64

In [5]:
df.culmen_length_mm.fillna(df.culmen_length_mm.mean(),inplace = True)
df.culmen_depth_mm.fillna(df.culmen_depth_mm.mean(),inplace = True)
df.flipper_length_mm.fillna(df.flipper_length_mm.mean(),inplace = True)
df.body_mass_g.fillna(df.body_mass_g.mean(),inplace = True)

In [6]:
df.sex.fillna(df.sex.mode()[0],inplace = True)

In [7]:
df.isnull().sum()

species              0
island               0
culmen_length_mm     0
culmen_depth_mm      0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

In [8]:
df.duplicated().sum()

0

In [9]:
inp = df.drop('species',axis = 1)
inp.head(2)

,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Torgersen,39.5,17.4,186.0,3800.0,FEMALE


In [10]:
X = pd.get_dummies(inp, columns=['island','sex'],drop_first = True).astype(int)
y = df['species']

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test  = train_test_split(X,y , random_state = 9 , test_size = 0.2)

## Modeling

In [12]:
estimator = RandomForestClassifier(random_state = 0) # lock the random state no for the fix the data 

param_grid = {'n_estimators':list(range(1,10))} # we cakn use bootstrap here 

grid = GridSearchCV(estimator,param_grid,cv = 5,scoring = 'accuracy')

grid.fit(X_train,y_train)

grid.best_params_

{'n_estimators': 9}

In [13]:
grid.best_estimator_

RandomForestClassifier(n_estimators=9, random_state=0)

In [14]:
grid.best_estimator_.feature_importances_

array([0.36675194, 0.13664949, 0.25849608, 0.13228838, 0.08098322,
       0.01840505, 0.00642584])

#### extract that feature which is more thenn importent 0 % and create new model

In [15]:
features = pd.DataFrame(data = grid.best_estimator_.feature_importances_,
                       index = X.columns,columns = ['Importance'])
features.head()

,Importance
culmen_length_mm,0.366752
culmen_depth_mm,0.136649
flipper_length_mm,0.258496
body_mass_g,0.132288
island_Dream,0.080983


In [16]:
imp_features = features[features['Importance']>0].index.tolist()
imp_features

['culmen_length_mm',
 'culmen_depth_mm',
 'flipper_length_mm',
 'body_mass_g',
 'island_Dream',
 'island_Torgersen',
 'sex_MALE']

In [17]:
X_imp = X[imp_features]

In [18]:
X_train,X_test,y_train,y_test  = train_test_split(X_imp,y , random_state = 9 , test_size = 0.2)

In [19]:
# model with best hyperparameter 
final_dt_model = dt_default = RandomForestClassifier(n_estimators = 7,random_state = 0)
final_dt_model.fit(X_train,y_train)

y_pred_train = final_dt_model.predict(X_train)
y_pred_test = final_dt_model.predict(X_test)

print('train aaccuracy',accuracy_score(y_train,y_pred_train))
print('test accuracy is :', accuracy_score(y_test,y_pred_test))
print('cross validation score is :',cross_val_score(final_dt_model,X_train,y_train,cv = 5).mean())

train aaccuracy 0.9963636363636363
test accuracy is : 0.9565217391304348
cross validation score is : 0.9781818181818182
